In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
data_set = pd.read_csv('/kaggle/input/human-date-to-machine-date/data_set.csv')
data_set[:10]

In [5]:
import tensorflow as tf

In [6]:
tf.__version__

In [7]:
model = tf.keras.models.Sequential()

In [8]:
set("dfdfsd ")

In [9]:
# creating vocabulary 
#human and machine
human_vocab = set()
machine_vocab = set()
for row in data_set.index:
    h, m = data_set.humanized[row].lower(), data_set.machine[row]
    human_vocab.update(tuple(h))
    machine_vocab.update(tuple(m))
human_vocab = dict(zip(sorted(human_vocab) + ['<unk>', '<pad>'], list(range(len(human_vocab) + 2))))
machine_vocab = dict(zip(sorted(machine_vocab), list(range(len(machine_vocab)))))
machine_vocab, len(machine_vocab), len(human_vocab)

In [10]:
# process data
# input pd [humanized, machine]
# output 
# X [list of list of words] 
# Y [list of list of words] 
# Xoh [list of list of dict of words] 
# Yoh [list of list of dict of works]

# Tx data_set.humanized.map(len).max() -> 27, lets take 30
# Ty data_set.machine.map(len).max() -> 10

X = []
Y = []
Tx = 30
Ty = 10
m = len(data_set)

def string_to_int(text, vocab, T=30):
    text = text.lower()
    if len(text) > T:
        text = text[:T]
    ints = [vocab.get(text[i]) if len(text) > i else vocab.get('<pad>') for i in range(T)]
    return ints

for i in data_set.index:
    hd, md = data_set.humanized[i], data_set.machine[i]
    X.append(string_to_int(hd, human_vocab, Tx))
    Y.append(string_to_int(md, machine_vocab, Ty))
X = np.array(X)
Y = np.array(Y)
Xoh = np.array(list(map(lambda x: tf.keras.utils.to_categorical(x, num_classes=len(human_vocab)), X)))
Yoh = np.array(list(map(lambda x: tf.keras.utils.to_categorical(x, num_classes=len(machine_vocab)), Y)))
X.shape, Y.shape, Xoh.shape, Yoh.shape, m

In [11]:
# one step attention
# input -> a [all activations of Bi LSTM] shape (30, 37)
# input -> s [t-1 hidden state of post attention LSTM] shape ()

def one_step_attention(a, s_prev, dense_1, dense_2):
    s_prev = tf.keras.layers.RepeatVector(Tx)(s_prev)
    con = tf.keras.layers.Concatenate(axis=-1)([a, s_prev])
    print('con', con.shape)
    dense1 = dense_1(con)
    dense2 = dense_2(dense1)
    energies = tf.keras.activations.softmax(dense2)
    context = tf.keras.layers.Dot(axes=1)([energies, a])
    return context
    
    
    
    

In [12]:
def model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):
    X = tf.keras.Input(shape=(Tx, human_vocab_size))
    s0 = tf.keras.Input(shape=(n_s,))
    c0 = tf.keras.Input(shape=(n_s,))
    s = s0
    c = c0
    
    outputs = []
    bilstm = tf.keras.layers.LSTM(n_a, return_sequences=True)
    a = tf.keras.layers.Bidirectional(bilstm)(X)
    print(a.shape)
    post_lstm = tf.keras.layers.LSTM(n_s, return_state=True)
    dense_1 = tf.keras.layers.Dense(10, activation='relu')
    dense_2 = tf.keras.layers.Dense(1, activation='relu')
    dense_out = tf.keras.layers.Dense(machine_vocab_size, activation='softmax')
    for t in range(Ty):
        context = one_step_attention(a, s, dense_1, dense_2)
        print(context.shape, 'context')
        s, _, c = post_lstm(context, initial_state=[s, c])
        out = dense_out(s)
        outputs.append(out)
    print(outputs[0].shape, Ty)
    return tf.keras.models.Model(inputs=[X, s0, c0], outputs=outputs)

sample_model = model(Tx, Ty, 32, 64, len(human_vocab), len(machine_vocab))
sample_model.summary()
    

In [31]:
sample_model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.005, beta_1=0.9, beta_2=0.999,decay=0.01), metrics=['accuracy'])

In [32]:
s0 = np.zeros((m, 64))
c0 = np.zeros((m, 64))
outputs = list(Yoh.swapaxes(0, 1))
print(outputs[0].shape)
sample_model.fit(x=[Xoh, s0, c0], y=outputs, batch_size=100, epochs=20)

In [17]:
EXAMPLES = ['3 May 1979', '5 April 09', '21th of August 2016', 'Tue 10 Jul 2007', 'Saturday May 9 2018', 'March 3 2001', 'March 3rd 2001', '1 March 2001']
machine_vocab_reversed = dict((v, k) for k, v in machine_vocab.items())
for example in EXAMPLES:
    numerized = string_to_int(example, human_vocab)
    one_hot = np.array([list(map(lambda x: tf.keras.utils.to_categorical(x, num_classes=len(human_vocab)), numerized))])
    prediction = sample_model.predict_step([one_hot, s0[:1, :], c0[:1, :]])
    pred_numerized = [np.argmax(x) for x in prediction]
    pred_alpha = [machine_vocab_reversed[x] for x in pred_numerized]
    print(''.join(pred_alpha))